In [1]:
!pip install selenium
!pip install webdriver-manager
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [21]:
!pip install findspark


Defaulting to user installation because normal site-packages is not writeable
  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import findspark
findspark.init()


In [23]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [24]:
url = 'https://www.cricbuzz.com/cricket-scorecard-archives'
driver.get(url)
time.sleep(3)

In [25]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [26]:
data = []
series_containers = driver.find_elements(By.CSS_SELECTOR, 'div.cb-col.cb-col-100.cb-bg-white')

print(f"Found {len(series_containers)} series containers.")

for container in series_containers:
    try:
        year_element = container.find_element(By.TAG_NAME, 'h2')
        year = year_element.text.strip()
        series_links = container.find_elements(By.TAG_NAME, 'a')

        for link in series_links:
            series_name = link.text.strip()
            series_href = link.get_attribute('href')
            data.append({
                'year': year,
                'series_name': series_name,
                'series_href': "https://www.cricbuzz.com" + series_href
            })

    except Exception as e:
        print(f"Error processing container: {e}")


Found 2 series containers.
Error processing container: Message: no such element: Unable to locate element: {"method":"tag name","selector":"h2"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B98073+60707]
	GetHandleVerifier [0x00B980B4+60772]
	(No symbol) [0x009C0683]
	(No symbol) [0x00A08660]
	(No symbol) [0x00A089FB]
	(No symbol) [0x009FDD81]
	(No symbol) [0x00A2D094]
	(No symbol) [0x009FDCA4]
	(No symbol) [0x00A2D2C4]
	(No symbol) [0x00A4E824]
	(No symbol) [0x00A2CE46]
	(No symbol) [0x009FC5D3]
	(No symbol) [0x009FD424]
	GetHandleVerifier [0x00DDBB53+2435075]
	GetHandleVerifier [0x00DD70F3+2416035]
	GetHandleVerifier [0x00DF349C+2531660]
	GetHandleVerifier [0x00BAF145+155125]
	GetHandleVerifier [0x00BB5AED+182173]
	GetHandleVerifier [0x00B9F948+91640]
	GetHandleVerifier [0x00B9FAF0+92064]
	GetHandleVerifier [0x

In [27]:
df = pd.DataFrame(data)
print(df.head())
df.to_csv('cricbuzz_series.csv', index=False)

print("Data saved successfully!")
driver.quit()

   year                               series_name  \
0  2025  Hong Kong T20I Quadrangular Series, 2025   
1  2025             Portugal tour of Norway, 2025   
2  2025              Canada tour of Namibia, 2025   
3  2025        Pakistan tour of New Zealand, 2025   
4  2025  Falkland Islands tour of Costa Rica 2025   

                                         series_href  
0  https://www.cricbuzz.comhttps://www.cricbuzz.c...  
1  https://www.cricbuzz.comhttps://www.cricbuzz.c...  
2  https://www.cricbuzz.comhttps://www.cricbuzz.c...  
3  https://www.cricbuzz.comhttps://www.cricbuzz.c...  
4  https://www.cricbuzz.comhttps://www.cricbuzz.c...  
Data saved successfully!


In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_extract, trim


In [29]:
spark = SparkSession.builder.appName("TieBreakerDeDuplication").getOrCreate()



ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
df = spark.read.option("header", True).csv("ipl_match_data.csv")


In [ ]:
spark_df.show(truncate=False)